# 📌 Detección de Tópicos con BERT (SBERT)
Este notebook permite subir un archivo Excel con ~200 documentos y aplicar **BERT + clustering** para descubrir tópicos.

### Pasos:
1. Subir tu archivo `.xlsx` o `.csv` con una columna `texto`.
2. Obtener embeddings con **SBERT**.
3. Clustering con **KMeans** o **HDBSCAN (UMAP)**.
4. Extraer palabras clave de cada tópico.
5. Guardar resultados a Excel.

In [ ]:
!pip install -q pandas openpyxl scikit-learn sentence-transformers umap-learn hdbscan matplotlib nltk

In [ ]:
from google.colab import files
import io, pandas as pd, re

print("➡️ Sube tu archivo DOCUMENTO200.xlsx con una columna 'texto'")
uploaded = files.upload()

fname = list(uploaded.keys())[0]
print("Archivo recibido:", fname)

if fname.lower().endswith(".xlsx"):
    df = pd.read_excel(io.BytesIO(uploaded[fname]))
elif fname.lower().endswith(".csv"):
    df = pd.read_csv(io.BytesIO(uploaded[fname]))
else:
    raise ValueError("Formato no soportado. Usa .xlsx o .csv")

assert "texto" in df.columns, "El archivo debe tener una columna 'texto'."
df = df.dropna(subset=["texto"]).copy()
df["texto"] = df["texto"].astype(str).str.strip()
df.head(30)

➡️ Sube tu archivo DOCUMENTO200.xlsx con una columna 'texto'


Saving documentos_200.xlsx to documentos_200 (1).xlsx
Archivo recibido: documentos_200 (1).xlsx


,id,texto,tema_esperado
0,1,El club fichó a un delantero tras la Copa Libe...,deportes_futbol
1,2,El índice bursátil cerró con ganancias por res...,finanzas_inversion
2,3,El técnico ajustó la táctica para presionar al...,deportes_futbol
3,4,La empresa lanzó un modelo BERT para análisis ...,tecnologia_ia
4,5,El índice bursátil cerró con ganancias por res...,finanzas_inversion
5,6,Fondos indexados ofrecen bajas comisiones y am...,finanzas_inversion
6,7,Observadores internacionales supervisarán la j...,politica_elecciones
7,8,La microbiota influye en el sistema inmunológi...,salud_nutricion
8,9,El programa de becas prioriza investigación ap...,educacion_universidades
9,10,Quito histórico destaca por su arquitectura co...,turismo_ecuador


In [ ]:
#aqui sacamos relaciones semanticas entre las palabras
from sentence_transformers import SentenceTransformer

def clean_txt(t):
    t = re.sub(r"\s+", " ", t)
    return t.strip()

df["texto_clean"] = df["texto"].apply(clean_txt)
texts = df["texto_clean"].tolist()

sbert = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
X = sbert.encode(texts, normalize_embeddings=True)
X.shape

(200, 384)

In [ ]:
# 🅰️ Enfoque A: KMeans
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import numpy as np

candidatos_k = list(range(5, 13))
scores = []
for k in candidatos_k:
    km = KMeans(n_clusters=k, n_init="auto", random_state=42)
    labels_tmp = km.fit_predict(X)
    sc = silhouette_score(X, labels_tmp)
    scores.append((k, sc))
best_k = max(scores, key=lambda x: x[1])[0]
print("Mejor K por silhouette:", best_k)

kmeans = KMeans(n_clusters=best_k, n_init="auto", random_state=42)
labels_km = kmeans.fit_predict(X)
df["topico_id_km"] = labels_km

tfidf = TfidfVectorizer(max_features=4000, ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(df["texto_clean"])
feat = np.array(tfidf.get_feature_names_out())

def top_keywords(labels, cluster_id, topn=10):
    # índices de los documentos que caen en el cluster indicado
    idx = np.where(labels == cluster_id)[0]
    if len(idx) == 0:
        return []

    # promedio del TF-IDF del cluster (sparse -> np.array 1D)
    row_mean = X_tfidf[idx].mean(axis=0)   # devuelve una matriz 1xN
    centroid = row_mean.A1                 # convierte a ndarray plano

    # normalización L2 (manual para no usar objetos matrix)
    norm = np.linalg.norm(centroid) + 1e-12
    centroid = centroid / norm

    # top N términos por peso en el "centroide" del cluster
    order = np.argsort(centroid)[-topn:][::-1]
    return feat[order].tolist()

topic_kw = {c: top_keywords(labels_km, c, topn=10) for c in sorted(np.unique(labels_km))}
topic_kw


Mejor K por silhouette: 12


{np.int32(0): ['en evaluación',
  'docentes capacitan',
  'evaluación por',
  'evaluación',
  'competencias',
  'capacitan',
  'capacitan en',
  'por competencias',
  'docentes',
  'caja'],
 np.int32(1): ['bienestar general',
  'dormir bien',
  'bienestar',
  'dormir',
  'bien es',
  'bien',
  'el bienestar',
  'para el',
  'es clave',
  'general'],
 np.int32(2): ['suministro',
  'suministro de',
  'glaciares afecta',
  'glaciares',
  'de glaciares',
  'el suministro',
  'agua',
  'afecta',
  'el derretimiento',
  'afecta el'],
 np.int32(3): ['por',
  'cuenca',
  'cultural',
  'patrimonio',
  'reconocida por',
  'reconocida',
  'su patrimonio',
  'es reconocida',
  'patrimonio cultural',
  'cuenca es'],
 np.int32(4): ['el',
  'con',
  'la',
  'en',
  'del',
  'electoral',
  'subieron por',
  'tesoro subieron',
  'de recesión',
  'del tesoro'],
 np.int32(5): ['inflación',
  'de referencia',
  'inflación impacta',
  'las tasas',
  'tasas de',
  'tasas',
  'de interés',
  'referencia',
  

In [ ]:
# 🅱️ Enfoque B: UMAP + HDBSCAN
import umap, hdbscan

umap_reducer = umap.UMAP(n_neighbors=15, n_components=5, min_dist=0.0, random_state=42) #aqui definimos que sean 5 topicos
X_umap = umap_reducer.fit_transform(X)

clusterer = hdbscan.HDBSCAN(min_cluster_size=8, min_samples=4, metric='euclidean', prediction_data=True)
labels_hdb = clusterer.fit_predict(X_umap)
df["topico_id_hdb"] = labels_hdb

df.head()

/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


,id,texto,tema_esperado,texto_clean,topico_id_km,topico_id_hdb
0,1,El club fichó a un delantero tras la Copa Libe...,deportes_futbol,El club fichó a un delantero tras la Copa Libe...,4,9
1,2,El índice bursátil cerró con ganancias por res...,finanzas_inversion,El índice bursátil cerró con ganancias por res...,3,4
2,3,El técnico ajustó la táctica para presionar al...,deportes_futbol,El técnico ajustó la táctica para presionar al...,4,5
3,4,La empresa lanzó un modelo BERT para análisis ...,tecnologia_ia,La empresa lanzó un modelo BERT para análisis ...,8,4
4,5,El índice bursátil cerró con ganancias por res...,finanzas_inversion,El índice bursátil cerró con ganancias por res...,3,4


In [ ]:
# 💾 Guardar y descargar resultados
outname = "resultados_topicos.xlsx"
df.to_excel(outname, index=False)
files.download(outname)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>